# Tests

## Vorspann

In [1]:
# wds_path="C:\\Users\\Georg\\Documents\\projekte\\WiagDataSetup.jl"
wds_path="..\\.."

"..\\.."

In [2]:
cd(wds_path)

In [3]:
pwd()

"C:\\Users\\georg\\Documents\\projekte\\WiagDataSetup.jl"

In [4]:
using Pkg

In [5]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [6]:
using Revise, MySQL

In [7]:
using WiagDataSetup

┌ Info: Precompiling WiagDataSetup [522c5ebb-a018-4020-8ed4-420cb1a9f084]
└ @ Base loading.jl:1423


In [8]:
using DataFrames

In [9]:
Wds = WiagDataSetup

WiagDataSetup

In [10]:
Wds.setDBWIAG(user="georg", db="wiag_in")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in")

## parse Datumsangaben
*2022-02-23* unscharfe Datumsangaben werden mit einer Reihe von Mustern interpretiert. Welche Angaben lassen sich nicht mit einem dieser Muster erfassen?

Ersetze die Warnmeldungen `@warn` mit `@assert`

### Domherrendatenbank

Lebensdaten

In [11]:
sql = "SELECT id, givenname, familyname, date_birth, date_death
FROM cn_canon 
WHERE status = 'online'";
df_canon = Wds.sql_df(sql);

In [20]:
parse_date_lower(x) = Wds.parsemaybe(x, :lower);
parse_date_upper(x) = Wds.parsemaybe(x, :upper);

In [24]:
for row in eachrow(df_canon)
    try
        parse_date_lower(row.date_birth)
    catch e
        if isa(e, AssertionError)
            println(e.msg, " ", row.id, " ", row.givenname, " ", row.familyname)
        else
            throw(e)
        end        
    end
end

In [ ]:
for row in eachrow(df_canon)
    try
        parse_date_upper(row.date_death)
    catch e
        if isa(e, AssertionError)
            println(e.msg, " ", row.id, " ", row.givenname, " ", row.familyname)
        else
            throw(e)
        end        
    end
end

Geburts- und Sterbedatum ergeben keine Fehler.

Amtsdaten

In [25]:
sql = "SELECT id, id_canon, office_name, date_start, date_end
FROM cn_office";
df_occ = Wds.sql_df(sql);

In [28]:
for row in eachrow(df_occ)
    try
        parse_date_lower(row.date_start)
    catch e
        if isa(e, AssertionError)
            println(e.msg, " ", row.id, " ", row.id_canon, " ", row.office_name)
        else
            throw(e)
        end        
    end
end

Could not parse unbekannt 10910 20277 Domdekan
Could only find year in von 1521 10916 20296 Archidiakon
Could not parse unbekannt 11033 20184 Propst
Could not parse wohl im 9. Jahrhundert 13084 13464 Dompropst
match doubtful: 11. oder 12. Jahrhundert-> 11 16281 25355 Domherr
match doubtful: 11. oder 12. Jahrhundert-> 11 16282 25351 Domherr
match doubtful: 11. oder 12. Jahrhundert-> 11 16311 24863 Domherr
match doubtful: 11. oder 12. Jahrhundert-> 11 16315 25453 Domherr
match doubtful: 11. oder 12. Jahrhundert-> 11 16316 25529 Domherr


SQL
```sql
SELECT id, id_canon, office_name, date_start
FROM cn_office
WHERE id in (10910, 10916, 11033, 13084, 16281, 16282, 16311, 16315, 16316)
```

In [30]:
for row in eachrow(df_occ)
    try
        parse_date_upper(row.date_end)
    catch e
        if isa(e, AssertionError)
            println(e.msg, " ", row.id, " ", row.id_canon, " ", row.office_name)
        else
            throw(e)
        end        
    end
end

### Personendatenbank/Digitales Personenregister

Lebensdaten

In [11]:
sql = "SELECT id, givenname, familyname, date_birth, date_death
FROM cn_canon_gs
WHERE status = 'online'";
df_canon = Wds.sql_df(sql);

In [12]:
size(df_canon, 1)

8564

In [20]:
parse_date_lower(x) = Wds.parsemaybe(x, :lower);
parse_date_upper(x) = Wds.parsemaybe(x, :upper);

In [33]:
for row in eachrow(df_canon)
    try
        parse_date_lower(row.date_birth)
    catch e
        if isa(e, AssertionError)
            println(e.msg, " ", row.id, " ", row.givenname, " ", row.familyname)
        else
            throw(e)
        end        
    end
end

In [36]:
for row in eachrow(df_canon)
    try
        parse_date_upper(row.date_death)
    catch e
        if isa(e, AssertionError)
            println(e.msg, " ", row.id, " ", row.givenname, " ", row.familyname)
        else
            throw(e)
        end        
    end
end

Could not parse 28. August 80647 Cuonradus missing
Could not parse Kammerkleriker 83297 Caspar Struick


SQL
``` sql
SELECT id, gsn_id, givenname, familyname, date_death
FROM cn_canon_gs
WHERE id in (80647, 83297)
```

Amtsdaten

In [37]:
sql = "SELECT id, id_canon, office_name, date_start, date_end
FROM cn_office_gs";
df_occ = Wds.sql_df(sql);

In [45]:
Wds.clean_up!(df_occ)

In [52]:
for row in eachrow(df_occ)
    try
        parse_date_lower(row.date_start)
    catch e
        if isa(e, AssertionError)
            println(e.msg, " ", row.id, " ", row.id_canon, " ", row.office_name)
        else
            throw(e)
        end        
    end
end

┌ Warning: Could only find year in 
│   s = um1220
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1097


SQL
```sql
SELECT o.id, o.id_canon, cn.gsn_id, office_name, date_start
FROM cn_office_gs AS o
JOIN cn_canon_gs AS cn ON cn.id = o.id_canon
WHERE o.id in (80831);
```

In [51]:
for row in eachrow(df_occ)
    try
        parse_date_upper(row.date_end)
    catch e
        if isa(e, AssertionError)
            println(e.msg, " ", row.id, " ", row.id_canon, " ", row.office_name)
        else
            throw(e)
        end        
    end
end

## neue Sortierung
*2022-02-23* siehe `C:\Users\georg\Documents\WIAG\WIAG-Technik\zeitlicheEingrenzung_sort.pdf`

In [11]:
using DataFrames

In [52]:
df_sample = DataFrame(
    date_string = [
        "vor 1200",
        "kurz vor 1200",
        "1200",
        "ca. 1200",
        "um 1200",
        "erstmals erwähnt 1200",
        "kurz nach 1200",
        "Anfang der 1200er Jahre",
        "nach 1200",
        "1200er Jahre",
        "Anfang 12. Jh.",
        "erstes Viertel des 12. Jhs.",
        "1. Hälfte des 12. Jhs.",
        "frühes 12. Jh.",
        "zweites Viertel des 12. Jhs.",
        "Mitte 12. Jh.",
        "drittes Viertel des 12. Jhs.",
        "2. Hälfte des 12. Jhs.",
        "spätes 12. Jh.",
        "Ende 12. Jh.",
        "viertes Viertel des 12. Jhs.",
        "12. Jahrhundert",
        "wohl im 12. Jahrhundert",
        "12. oder 13. Jahrhundert"
    ]
);

In [53]:
transform!(df_sample, :date_string => ByRow(Wds.parse_year_sort) => :sort);

In [54]:
df_sample

,date_string,sort
,String,Int64
1,vor 1200,1200100
2,kurz vor 1200,1200105
3,1200,1200150
4,ca. 1200,1200200
5,um 1200,1200210
6,erstmals erwähnt 1200,1200110
7,kurz nach 1200,1200303
8,Anfang der 1200er Jahre,1200305
9,nach 1200,1200309


## parse Datumsangaben zur Sortierung
*2022-02-23* unscharfe Datumsangaben werden mit einer Reihe von Mustern interpretiert. Welche Angaben lassen sich nicht mit einem dieser Muster erfassen?

Ersetze die Warnmeldungen `@warn` mit `@assert`

### Domherrendatenbank

Lebensdaten

In [46]:
sql = "SELECT id, givenname, familyname, date_birth, date_death
FROM cn_canon 
WHERE status = 'online'";
df_canon = Wds.sql_df(sql);

In [47]:
for row in eachrow(df_canon)
    try
        Wds.parse_year_sort(row.date_birth)
        Wds.parse_year_sort(row.date_death)
    catch e
        if isa(e, AssertionError)
            println(e.msg, " ", row.id, " ", row.givenname, " ", row.familyname)
        else
            throw(e)
        end        
    end
end

could not parse (nach) 1317 20463 Konard Valkenstein
could not parse (vor) 1346 26583 Ludolf Wohldenberg


Amtsdaten

In [32]:
sql = "SELECT id, id_canon, office_name, date_start, date_end
FROM cn_office";
df_occ = Wds.sql_df(sql);

In [34]:
Wds.clean_up!(df_occ)

In [35]:
for row in eachrow(df_occ)
    try
        Wds.parse_year_sort(row.date_start)
        Wds.parse_year_sort(row.date_end)
    catch e
        if isa(e, AssertionError)
            println(e.msg, " ", row.id, " ", row.id_canon, " ", row.office_name)
        else
            throw(e)
        end        
    end
end

could not parse von 1521 10916 20296 Archidiakon
could not parse (vor) 1464 12498 13672 Domherr


In [30]:
for row in eachrow(df_occ)
    try
        parse_date_upper(row.date_end)
    catch e
        if isa(e, AssertionError)
            println(e.msg, " ", row.id, " ", row.id_canon, " ", row.office_name)
        else
            throw(e)
        end        
    end
end

### Personendatenbank/Digitales Personenregister

Lebensdaten

In [39]:
sql = "SELECT id, givenname, familyname, date_birth, date_death
FROM cn_canon_gs
WHERE status = 'online'";
df_canon = Wds.sql_df(sql);

In [40]:
Wds.clean_up!(df_canon)

In [41]:
size(df_canon, 1)

8396

In [42]:
for row in eachrow(df_canon)
    try
        Wds.parse_year_sort(row.date_birth)
        Wds.parse_year_sort(row.date_death)
    catch e
        if isa(e, AssertionError)
            println(e.msg, " ", row.id, " ", row.givenname, " ", row.familyname)
        else
            throw(e)
        end        
    end
end

could not parse 28. August 80647 Cuonradus missing
could not parse Kammerkleriker 83297 Caspar Struick


Amtsdaten

In [13]:
sql = "SELECT id, id_canon, office_name, date_start, date_end
FROM cn_office_gs";
df_occ = Wds.sql_df(sql);

In [14]:
Wds.clean_up!(df_occ)

In [15]:
for row in eachrow(df_occ)
    try
        Wds.parse_year_sort(row.date_start)
        Wds.parse_year_sort(row.date_end)
    catch e
        if isa(e, AssertionError)
            println(e.msg, " ", row.id, " ", row.id_canon, " ", row.office_name)
        else
            throw(e)
        end        
    end
end

## Debug

In [61]:
Wds.parse_year_sort("11./12. Jahrhundert")

1099850

In [63]:
Wds.parsemaybe("11. oder 12. Jahrhundert", :lower)

1001